In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# better display of review text in dataframes
pd.set_option('display.max_colwidth', None)

# Seaborn options
sns.set(style="whitegrid", font_scale=1.4)

# Auto reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

TypeError: 'NoneType' object is not callable

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import requests

# Load RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Tokenization
tokenized_texts = [tokenizer.encode(text, max_length=512, truncation=True) for text in df['Texte']]

In [ ]:
import pandas as pd
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import requests

# Set the timeout value in seconds
timeout_value = 1000000000


# Load RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Tokenization
tokenized_texts = [tokenizer.encode(text, max_length=512, truncation=True) for text in df['Texte']]

# Perform sentiment analysis
results = []
for tokenized_text in tokenized_texts:
    inputs = tokenizer(tokenized_text, return_tensors='pt', padding=True, truncation=True)
    try:
        with requests.Session() as session:
            response = session.post("https://api-inference.huggingface.co/models/",
                                    json=inputs, timeout=timeout_value)
            response.raise_for_status()
            outputs = response.json()
    except requests.exceptions.Timeout:
        print("Request timed out")
        continue
    logits = outputs['logits']
    predicted_class = logits.argmax().item()
    results.append(predicted_class)

# Post-processing
sentiments = ['negative', 'neutral', 'positive']
df['Sentiment'] = [sentiments[result] for result in results]

# Analysis and visualization
sentiment_counts = df['Sentiment'].value_counts()
print(sentiment_counts)
# You can further analyze or visualize sentiment distribution as per your requirement

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

### Camembert

In [11]:
import torch

from transformers.modeling_camembert import CamembertForMaskedLM
from transformers.tokenization_camembert import CamembertTokenizer


def fill_mask(masked_input, model, tokenizer, topk=5):
    # Adapted from https://github.com/pytorch/fairseq/blob/master/fairseq/models/roberta/hub_interface.py
    assert masked_input.count("<mask>") == 1
    input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    logits = model(input_ids)[0]  # The last hidden-state is the first element of the output tuple
    masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    logits = logits[0, masked_index, :]
    prob = logits.softmax(dim=0)
    values, indices = prob.topk(k=topk, dim=0)
    topk_predicted_token_bpe = " ".join(
        [tokenizer.convert_ids_to_tokens(indices[i].item()) for i in range(len(indices))]
    )
    masked_token = tokenizer.mask_token
    topk_filled_outputs = []
    for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(" ")):
        predicted_token = predicted_token_bpe.replace("\u2581", " ")
        if " {0}".format(masked_token) in masked_input:
            topk_filled_outputs.append(
                (
                    masked_input.replace(" {0}".format(masked_token), predicted_token),
                    values[index].item(),
                    predicted_token,
                )
            )
        else:
            topk_filled_outputs.append(
                (masked_input.replace(masked_token, predicted_token), values[index].item(), predicted_token,)
            )
    return topk_filled_outputs


tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")
model.eval()

masked_input = "Le camembert est <mask> :)"
print(fill_mask(masked_input, model, tokenizer, topk=3))

ModuleNotFoundError: No module named 'transformers.modeling_camembert'